# Lightgbm

In [ ]:
!pip install bayesian-optimization

In [1]:
import pandas as pd
import numpy as np
import scipy as sp

from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.svm import SVC, SVR
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from bayes_opt import BayesianOptimization  # bayesian-optimization

from sklearn.metrics import mean_squared_error, accuracy_score

import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

# Hyperparameter Tuning

## Manual Search

- 직접 파라미터를 설정하는 직접 탐색 방법  
- 전체 범위를 보는 것이 아니기에 현재까지의 최적의 파라미터가 최적인지는 보장하지 못 함  
- 여러 종류의 파라미터를 동시에 탐색하기에는 한계

## Greed Search

<img src="https://www.yourdatateacher.com/wp-content/uploads/2021/03/image-6.png" width="400" height="400"/>

- 탐색 구간 내 모든 hyperparameter 조합을 일정 구간으로 나누어 시도  
- 파라미터가 많아질수록 기하급수적으로 많은 시간 소요

<br>

- 사용 방법
> ```python
> from sklearn.model_selection import GridSearchCV
> parameters = {'parameter1': ('value1', 'value2', ...), 'parameter2':[value1, value2, ...]}
> grid_search = GridSearchCV(model, parameters)
> grid_search.fit(X, y)
> ```

## Random Search

<img src="https://www.yourdatateacher.com/wp-content/uploads/2021/03/image-7.png" width="400" height="400"/>

- 탐색 대상 구간 내의 hyperparameter를 랜덤 샘플링  
- Grid Search 대비 반복 횟수를 줄이는 동시에 확률적 탐색으로 **최적에 근사한 parameter를  빨리 찾을 수 있는 것으로 알려짐**  
- 다만, 전체를 확인하는 게 아니라 최적의 값은 아님

<br>

- 사용 방법
> ```python
> from sklearn.model_selection import RandomizedSearchCV
> from scipy.stats import uniform
> distributions = {'parameter1': ('value1', 'value2', ...), 'parameter2': uniform(loc=1, scale=10)}
> grid_search = RandomizedSearchCV(model, parameters, random_state=0)
> grid_search.fit(X, y)
> ```

## Bayesian Optimization

<img src="https://miro.medium.com/v2/resize:fit:720/format:webp/1*PhKGj_bZlND8IEfII426wA.png" width="600" height="400"/>


- 탐색 대상 구간 내의 hyperparameter를 샘플링  
- Grid Search나 Random Search의 경우 다음 샘플링 선정 시 이전 샘플링의 정보를 사용하지 못하여 불필요한 탐색을 반복  
- Bayesian Optimization은 사전 정보를 활용하여 다음 sample의 후보군을 선택  
순차적으로 하이퍼파라미터를 업데이트해가면서 평가를 통해 최적의 하이퍼파라미터 조합 탐색

<br>

- 사용 방법

> ```python
> import numpy as np
> import lightgbm
> from sklearn.metrics import mean_squared_error
> from bayes_opt import BayesianOptimization
> 
> def lgbm_cv(
>   max_depth,
>   learning_rate,
>   n_estimators,
>   subsample,
>   colsample_bytree
>   ):
> 
>   # model define
>   model = lightgbm.LGBMRegressor(
>               max_depth=int(max_depth),
>               learning_rate=learning_rate,
>               n_estimators=int(n_estimators),
>               subsample=subsample,
>               colsample_bytree=colsample_bytree,
>           )
>
>   # train
>   model.fit(X_train, y_train)
> 
>   # predict
>   y_pred = model.predict(X_test)
> 
>   # metric
>   rmse = np.sqrt(mean_squared_error(y_test, y_pred))
> 
>   # metric return
>   return rmse
> ```

> ```python
> pbounds = {
>     'max_depth': (3, 10),
>     'learning_rate': (0.001, 0.1),
>     'n_estimators': (10, 1000),
>     'subsample': (0.5, 1),
>     'colsample_bytree' :(0.2, 1),
> }
> 
> bayesian_optimization = BayesianOptimization(
>     f=lgbm_cv,
>     pbounds=pbounds,
>     verbose=2,          # 출력 옵션
>     random_state=0,     # 2: 출력, 1: 최댓값일 때 출력, 0: 출력 안 함
>     )
> 
> bayesian_optimization.maximize(init_points=2, n_iter=10, acq='ei', xi=0.01)
> # init_points: 초기 random search 수
> # n_iter: 반복 횟수
> # acq: acquisition function
> # xi: exploration 강도
> print(bayesian_optimization.max)
> ```